# Battle of Neighbourhoods - Week 1

    This notebook consist of two parts.

#### 1 Introduction/Business Problem

    1.a Discussion of the business problem and the audience who would be interested in this project.

#### 2 Data Section

    2.a What data is used?
    2.b Importing Libraries
    2.c Credentials and Core location
    2.d Search for hotel & restaurant within 1 KM
    2.e Location of Hotels

## 1. Introduction/Business Problem

#### Discussion of the business problem and the audience who would be interested in this project.

##### Something about the tourist place - sharm el sheikh,Egypt

    it is an Egyptian city on the southern tip of the Sinai Peninsula, in South Sinai Governorate, on the coastal strip along the Red Sea. Its population is approximately 73,000 as of 2015. Sharm El Sheikh is the administrative hub of Egypt's South Sinai Governorate, which includes the smaller coastal towns of Dahab and Nuweiba as well as the mountainous interior, St. Catherine and Mount Sinai. The city and holiday resort is a significant centre for tourism in Egypt, while also attracting many international conferences and diplomatic meetings.

#### Opening of Hotel/Restaurant Shop

    Coming down to business problem,I would like to open a hotel/restaurant near beach side.As it is a famous tourist spot,there is already lots of attention towards it.I know there will be many competitors in terms of hotel and restauramt.But keeping them in mind,i need to locate my hotel in place where more people are attracted and comfortable for a stay and a good meal.I want to bring foreign and local peoples attention towards my new hotel.I would like to flavour my retaurant recipe with Italian,American,typical Egyptian foods to grab their taste.

    The challenge is to find a suitable location for opening a new hotel / restaurant attracted to all local and foreign people in the centre of all famous venues.

#### Expected / Interested Audience

Well, almost every business man who wnats to open a hotel in sharm el sheikh and needs a good place srounded by lots of nice place that local people and tourists are interseted in it like parks, cafes, dive shops and etc.

## 2.Data section

#### 2.a What data is used?

    We will be completely working on Foursquare data to explore and try to locate our new hotel where more venues like church, beach, museums, memorials that are present nearby.
#### How will we be solving using this data?

    We will look for midpoint area of venues to locate our new hotel.Before that our major focus will be on all venues present in and around the core place of sharm el sheikh.

    Just a heads up on how many hotels are distributed now around sharm el sheikh.We will perform some EDA on hotels & restaurants present in the tourist spot.On furthur notebook we will use Foursquare data to determine other venues as well.

#### 2.b Importing Libraries

In [3]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
!pip install beautifulsoup4
# Import libraries
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # data analsysis
import requests # HTTP library
from bs4 import BeautifulSoup # scraping library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation

from sklearn.cluster import KMeans # clustering algorithm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt


from geopy.geocoders import Nominatim

import folium # map rendering library

print("*********   done!     ***********")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

     |████████████████████████████████| 115 kB 3.3 MB/s eta 0:00:01
*********   done!     ***********


#### 2.c Credentials and Core location

In [4]:

CLIENT_ID = '23C50AA4MZYMOXVM5Q4ZMZNFI1MFXQHRRD0NLJAJU1MZU1YE' # your Foursquare ID
CLIENT_SECRET = 'FBR5LKO0XY34LSXEI4QMX5ZVHCUCFYHDEXIUP1ZJWO12PPZS' # your Foursquare Secret
VERSION = '20191028'
LIMIT = 150

address = "sharm el sheikh, south sinai"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude =location.latitude   #8.079252 # location.latitude 
longitude =location.longitude #77.5499338 # location.longitude # 

sharm='sharm el sheikh location : {},{}'.format(latitude,longitude)
print(sharm)

sharm el sheikh location : 27.8669082,34.3014551


#### 2.d Search for hotel & restaurant within 1 KM

In [5]:
#Quering for hotel & restaurant

search_query = 'hotel'
search_query_res = 'restaurant'

radius = 1000
url_hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_restaurant = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_res, radius, LIMIT)
#url

##### Send the GET Request of hotel & restaurants and examine the results

In [6]:
results_hotel = requests.get(url_hotel).json()
results_restaurant = requests.get(url_restaurant).json()
#results_hotel

##### Get relevant part of JSON and transform it into a pandas dataframe

In [7]:
# assign relevant part of JSON to venues
venues_hotel = results_hotel['response']['venues']
venues_restaurant = results_restaurant['response']['venues']

# tranform venues into a dataframe and merging both data
dataframe_hotel = json_normalize(venues_hotel)
dataframe_restaurant = json_normalize(venues_restaurant)

dataframe = pd.concat([dataframe_hotel,dataframe_restaurant])

print("There are {} restaurants and hotels at sharm el sheikh".format(dataframe.shape[0]))

There are 41 restaurants and hotels at sharm el sheikh


##### Define information of interest and filter dataframe

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

  
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered
hotels_df=dataframe_filtered[['name','categories','distance','lat','lng','id']]
hotels_df.head()

,name,categories,distance,lat,lng,id
0,Aida Hotel,Resort,232,27.865193,34.302795,4f7df6cae4b02c4144246ec1
1,Reception at Beach Albatros Hotel,Travel Lounge,839,27.859548,34.299601,52eb96a7498ead9f84d77ec8
2,Bar Club House at Beach Albatros Hotel,Beach Bar,914,27.858958,34.299093,52eeca4b11d2b781e6bcf21c
3,Pool at Aida Hotel,Pool,230,27.864835,34.301360,520d42d811d2869e9eec26fb
4,Renissance Hotel Spa,Spa,232,27.866892,34.303822,4ffc674ce4b07354de04550e


#### 2.e Location of Hotels

In [10]:
hotels_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the Kanyakumari

# add a red circle marker to represent the core location of kanyakumari
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='sharm el sheikh',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(hotels_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(hotels_df.lat, hotels_df.lng, hotels_df.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)

# display map
hotels_map

# that is it for the first part